In [1]:
import pandas as pd

boardbook = pd.read_csv("./premier_boardbook_schools.csv")
columns = ['SchoolName', 'Index']
boardbook = boardbook[columns]

boardbook.columns = ["school_name", "index"]

boardbook['school_name'] = boardbook['school_name'].str.replace(' SD', '', case=False)
boardbook["school_name"] = boardbook["school_name"].str.lower().str.replace(" ", "")
boardbook['school_name'] = boardbook['school_name'].str.replace('schooldistrict', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('schooldist', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('district', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('publicschools', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('publicschool', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('countyschools', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('countyschool', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('county', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('schools', '', case=False)
boardbook['school_name'] = boardbook['school_name'].str.replace('school', '', case=False)

boardbook

,school_name,index
0,100-297hsgrant,3184
1,a+charter,2823
2,abernathyisd,1111
3,abileneindependent,2274
4,academyisd,2299
...,...,...
1711,yukonflats,2391
1712,yumaelementaryone,2085
1713,zapataindependent,2305
1714,zionelementary6,1476


In [2]:
import pandas as pd

nces = pd.read_csv("../nces2324.csv")

columns = ['LEAID', 'LEA_NAME']
nces = nces[columns]

nces.columns = ["LEAID", "school_name"]


nces['school_name'] = nces['school_name'].str.replace(' SD', '', case=False)
nces["school_name"] = nces["school_name"].str.lower().str.replace(" ", "")
nces['school_name'] = nces['school_name'].str.replace('schooldistrict', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('schooldist', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('district', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('publicschools', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('publicschool', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('countyschools', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('countyschool', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('county', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('schools', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('school', '', case=False)




nces

/var/folders/k7/0d0pjdr93h15578mwngc_klc0000gn/T/ipykernel_6309/2103092195.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nces = pd.read_csv("../nces2324.csv")


,LEAID,school_name
0,100002,alabamayouthservices
1,100005,albertvillecity
2,100006,marshall
3,100007,hoovercity
4,100008,madisoncity
...,...,...
19632,6600041,careertechhighacademycharter
19633,6900002,commonwealthofthenorthernmarianaislands
19634,7200030,puertoricodepartmentofeducation
19635,7800002,saintcroix


In [3]:
import pandas as pd

# Removed duplicate names
nces = nces.drop_duplicates(subset='school_name')

# Merge on school_name (case insensitive for better match rate)
merged_df = boardbook.merge(
    nces, 
    how='left', 
    on='school_name'  # change to case-insensitive matching below if needed
)

merged_df['LEAID'] = merged_df['LEAID'].astype('Int64').astype('string')

# Save the result to a new CSV
merged_df.to_csv('premier_merged_boardbook_with_leaid.csv', index=False)

# Count non-null LEAID values
not_null_count = merged_df['LEAID'].notna().sum()

# Total number of rows in boardbook
total_rows = len(boardbook)

# Calculate percentage
percentage = (not_null_count / total_rows) * 100

# Print results
print(f"Number of matches (non-null LEAID): {not_null_count}")
print(f"Total rows: {total_rows}")
print(f"Percentage matched: {percentage:.2f}%")

merged_df

merged_df.to_csv('merged_boardbook_with_leaid.csv', index=False)

Number of matches (non-null LEAID): 910
Total rows: 1716
Percentage matched: 53.03%


In [1]:
import csv

# Load LEAID data into a dictionary: index -> LEAID
leaid_lookup = {}
with open('premier_merged_boardbook_with_leaid.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        leaid_lookup[row['index']] = row['LEAID']

# Read first CSV and append LEAID if index matches
merged_rows = []
with open('filtered_schools.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        index = row['Index']
        row['LEAID'] = leaid_lookup.get(index, '')  # default empty if not found
        merged_rows.append(row)

# Write output to new CSV
with open('filtered_schools_with_leaid.csv', 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['School Name', 'Index', 'LEAID']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(merged_rows)

print("Merged data written to 'filtered_schools_with_leaid.csv'")

Merged data written to 'filtered_schools_with_leaid.csv'
